In [1]:
!pip install -U -q PyDrive
#!pip install pygeoip
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import socket
import re
import csv
import urllib.request
from urllib.request import urlopen
import urllib3
from xml.dom import minidom
#from pygeoip import *
from html.parser import HTMLParser
from collections import defaultdict

Python Selenium

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#https://drive.google.com/file/d/1ViPzkX9gXGrK6oUAVNdXxC7rQ51BRFjH/view?usp=sharing
fileDownloaded = drive.CreateFile({'id': '1ViPzkX9gXGrK6oUAVNdXxC7rQ51BRFjH'})

In [4]:
fileDownloaded.GetContentFile("url_classification.csv")
cat_df = pd.read_csv("url_classification.csv")
print(f'No. of categories: {len(set(cat_df.Category))}')
print(f'Categories are : {set(cat_df.Category)}')
print(f'Total no. of records: {len(cat_df)}')

No. of categories: 12
Categories are : {'Health', 'Business', 'Science', 'Adult', 'Home', 'News', 'Arts', 'Games', 'Reference', 'Kids', 'Computers', 'Recreation'}
Total no. of records: 1048575


In [5]:
def url_tokenize(url):
    tokenized_word = re.split('\W+', url)
    num_element, sum_of_element, largest = 0, 0, 0
    for element in tokenized_word:
        l = len(element)
        sum_of_element += l
        if l>0:
            num_element+=1
            if largest<l:
                largest=l
    try:
        return ([float(sum_of_element)/num_element, num_element, largest])
    except:
        return ([0, num_element, largest])

In [6]:
def url_has_exe(url):
    try:
        if url.find('.exe/')!=-1:
            return [url, 1]
    except:
        ;

In [7]:
def phishing_word_count(url):
    tokenized_word = re.split('\W+', url)
    phishing_words = ['confirm', 'account', 'banking', 'secure', 'rolex', 'login', 'signin', 'delete', 'lucky', 'free', 'archive', 'save']
    words=[]
    count=0
    for element in phishing_words:
        if element in tokenized_word:
            count+=1
            words.append(element)
    if count>0:
        return count, words

In [9]:
def scan_pg_src(url):
    class MyHTMLParser(HTMLParser):
        def __init__(self):
            self.count = defaultdict(int)
            super().__init__()

        def handle_starttag(self, tag, attrs):
            self.count[tag] += 1

        def handle_startendtag(self, tag, attrs):
            self.count[tag] += 1

    def count_tags(html):
        parser = MyHTMLParser()
        parser.feed(html)
        return parser.count

    wd = webdriver.Chrome('chromedriver',options=chrome_options)
    wd.get("http://www.liquidgeneration.com/")
    #print(wd.page_source)
    return count_tags(wd.page_source)

Finding presence of .exe in URL

In [10]:
for i in cat_df.values:
    if url_has_exe(i[1])==None:
        continue
    else: print(url_has_exe(i[1]))

['http://www.brb.org.uk/webpack/bin/webpack.exe/livebase?object=livebase1&amp;itemurn=1190&amp;mode=wbfullitem', 1]
['http://www.astm.org/cgi-bin/softcart.exe/commit/committee/f17.htm?l+mystore+zrkn6860+947885479', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/c1208c1208m.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/f1867.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/f1947.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/f1216.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/d5813.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/d1784.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/f1533.htm?l+mystore+ifma3146', 1]
['http://www.astm.org/cgi-bin/softcart.exe/database.cart/pages/f1743.htm?l+mystore+ifma3146', 1]


Finding presence of common phishing words

In [11]:
for i in cat_df.values:
    try:
        if phishing_word_count(i[1])==None:
            continue
        print(i[1], phishing_word_count(i[1]))
    except:
        print(i[1])

Streaming output truncated to the last 5000 lines.
http://www.karas-adult-playground-karasxxx.com/dabbers-free-toy-sex-pics.htm (1, ['free'])
http://www.pussey-free-porn-pics-free-porn-pics-free-porn-pics.com/ever-toys-free-sex/ (1, ['free'])
http://www.free-sex-pictures-free-sex-pictures.com/extreme-dildo-fuck/ (1, ['free'])
http://www.onlinesexhq.com/eacans-free-toy-sex.htm (1, ['free'])
http://www.teens-pics-free-porn-pussy-free-porn.com/free-pussy-pix/ (1, ['free'])
http://www.free-teen-pics-free-teen-pics.com/free_porn/index.html (1, ['free'])
http://www.e-free-porn-pics-free-porn-pics-free-porn-pics.com/free-dildo-pictures/ (1, ['free'])
http://www.fetish-free-porn-pics.com/ (1, ['free'])
http://www.hot-free-nude-pussy-pics-and-sex-videos.com/free_hardcore_toy_pix/ (1, ['free'])
http://www.free-sex-pictures-free-sex-pictures.com/free-dildo-fetish-sex/ (1, ['free'])
http://www.1-free-sex-free-sex-free-sex.com/free-pussy-sex/ (1, ['free'])
http://www.quality-free-xxx-adult-sex-porn

Tokenizing the URL -- avg token len, #of tokens, token with largest len

In [12]:
for i in cat_df.head(100).values:
    try:
        if url_tokenize(i[1])==None:
            continue
        print(i[1], url_tokenize(i[1]))
    except:
        print(i[1])

http://www.liquidgeneration.com/ [6.5, 4, 16]
http://www.onlineanime.org/ [5.25, 4, 11]
http://www.ceres.dti.ne.jp/~nekoi/senno/senfirst.html [4.1, 10, 8]
http://www.galeon.com/kmh/ [3.8, 5, 6]
http://www.fanworkrecs.com/ [5.25, 4, 11]
http://www.animehouse.com/ [5.0, 4, 10]
http://www2.117.ne.jp/~mb1996ax/enadc.html [4.0, 8, 8]
http://archive.rhps.org/fritters/yui/index.html [4.75, 8, 8]
http://www.freecartoonsex.com/ [6.0, 4, 14]
http://www.cutepet.org/ [4.25, 4, 7]
http://www.taremeparadise.com/ [6.0, 4, 14]
http://www.internetdump.com/users/pornographite/index1.html [6.25, 8, 13]
http://darkkaminari.net [6.333333333333333, 3, 12]
http://www.iei.net/~bkos1/velneko.htm [4.0, 7, 7]
http://www9.kinghost.com/fetish/hentaibee/ [5.666666666666667, 6, 9]
http://www.jasonmeador.com/ [5.25, 4, 11]
http://www.geocities.com/kaseychan17/index.html [5.571428571428571, 7, 11]
http://www.angelfire.com/journal/coldlemonade/index.html [5.875, 8, 12]
http://e.webring.com/hub?sid=&amp;ring=hentff98&am

Extracting page source. Counting different HTML tags

In [13]:
for i in cat_df.head().values:
    try:
        if scan_pg_src(i[1])==None:
            continue
        print(i[1], scan_pg_src(i[1]))
    except:
        print(i[1])

http://www.liquidgeneration.com/ defaultdict(<class 'int'>, {'html': 1, 'head': 1, 'meta': 41, 'link': 5, 'title': 1, 'script': 30, 'body': 1, 'div': 80, 'ins': 10, 'iframe': 13, 'a': 18, 'img': 7, 'h1': 6, 'span': 5, 'p': 5, 'br': 11, 'ul': 1, 'li': 4})
http://www.onlineanime.org/ defaultdict(<class 'int'>, {'html': 1, 'head': 1, 'meta': 41, 'link': 4, 'title': 1, 'script': 29, 'body': 1, 'div': 80, 'ins': 10, 'iframe': 13, 'a': 18, 'img': 7, 'h1': 6, 'span': 5, 'p': 5, 'br': 11, 'ul': 1, 'li': 4})
http://www.ceres.dti.ne.jp/~nekoi/senno/senfirst.html defaultdict(<class 'int'>, {'html': 1, 'head': 1, 'meta': 41, 'link': 4, 'title': 1, 'script': 29, 'body': 1, 'div': 80, 'ins': 10, 'iframe': 13, 'a': 18, 'img': 7, 'h1': 6, 'span': 5, 'p': 5, 'br': 11, 'ul': 1, 'li': 4})
http://www.galeon.com/kmh/ defaultdict(<class 'int'>, {'html': 1, 'head': 1, 'meta': 41, 'link': 5, 'title': 1, 'script': 30, 'body': 1, 'div': 80, 'ins': 10, 'iframe': 13, 'a': 18, 'img': 7, 'h1': 6, 'span': 5, 'p': 5,